# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [7]:
# 1) Seu código aqui
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Estado'

r = requests.get(url)
data_json = r.json()
dados = pd.DataFrame(data_json['items'])
dados.head(3)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,13483,Estado,Rio de Janeiro,33,RJ,None,Deferido,19405.000010/2004-52,2006-02-15T02:00:00Z,Operação contratual interna,Renegociação de dívidas,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,8.475263e+08,1,0,01/03/2006
1,9102,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000903/2009-80,2009-07-29T03:00:00Z,Operação contratual interna (com garantia da U...,PEF - Programa Emergencial de Financiamento do...,Instituição Financeira Nacional,Banco do Brasil S/A,Real,6.110800e+07,1,0,06/08/2009
2,8528,Estado,Rio de Janeiro,33,RJ,None,Deferido,17944.000793/2009-56,2009-07-17T03:00:00Z,Operação contratual interna,"Aquisição de terrenos, imóveis e/ou construção...",Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,1.570000e+08,1,0,24/07/2009


In [6]:
# 2) Seu código aqui
dados['status'].value_counts().head(3)

Encaminhado à PGFN com manifestação técnica favorável    34
Deferido                                                 14
Arquivado a pedido                                        3
Name: status, dtype: int64

In [62]:
# 3) Seu código aqui
dados['data_status'] = pd.to_datetime(dados['data_status'], format='%d/%m/%Y')
dados['ano_status'] = dados['data_status'].dt.year
dados['ano_status'].value_counts()

2012    11
2009     9
2013     7
2011     6
2014     4
2002     4
2016     4
2010     3
2006     2
2008     2
Name: ano_status, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [152]:
#1) Seu código aqui
def cria_url(uf: str, tipo: str) -> pd.DataFrame:
    
    url = f'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={uf}&tipo_interessado={tipo}'
    
    r = requests.get(url)
    data_json = r.json()
    dados = pd.DataFrame(data_json['items'])
    
    return dados

In [163]:
# 2) Seu código aqui
dados_mg = cria_url('MG','Estado')
len(dados_mg.loc[dados_mg['status'] == 'Arquivado por decurso de prazo'])

1

In [170]:
# 3) Seu código aqui
dados_ba = cria_url('BA','Município')
dados_ba['interessado'].value_counts().head(1)

Camaçari    19
Name: interessado, dtype: int64

In [172]:
# 4) Seu código aqui
dados_ba_2 = cria_url('BA','Estado')
dados_ba_2.to_csv('Dados_ba_2.csv')